In [0]:
#Final delivery 3
#https://www.kaggle.com/datasets/georgejnr/used-and-new-cars-datasets
#Predicting price of Cars Sales
#Tyler Rappaport, Yanjing Wang, Ziru Wang
#Ban 5600: Advanced Big Data Computing and Programming 
#Prof. Hamidreza Ahady Dolatsara
#April 18, 2023

#Environment Preparation

In [0]:
pip install pysparkling

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
pip install h2o

Python interpreter will be restarted.
Python interpreter will be restarted.


<div style="background-color:#F7CAC9; color:#4A3C31; padding:10px;">
    
# Data Understanding



</div>

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [0]:
spark=SparkSession.builder.appName('Used-Car-Price').getOrCreate()

In [0]:
#Understanding Data Types
#below are three types of loading the data, with the first two being for databricks, and the later for the local machine. The later method would need to be
#changed to work with the specific users machine
#df=spark.read.format("delta").load('/user/hive/warehouse/car_data',header=True, inferSchema=True)
df= spark.read.csv('/FileStore/tables/car_data.csv',inferSchema=True,header=True)
#df= spark.read.csv('file:///C:/Users/tyrap/Desktop/car_data.csv',inferSchema=True,header=True)
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Model: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Status: string (nullable = true)
 |-- Mileage: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- MSRP: string (nullable = true)



In [0]:
#Calculate the number of NA
columns = df.columns

for col in columns:
    null_count = df.filter(df[col].isNull()).count()
    print(f"Column '{col}' has {null_count} null values.")
    
total_null_count = df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).first()
total_null_count_dict = total_null_count.asDict()

print(f"Total null values in the dataframe: {total_null_count}")


Column '_c0' has 0 null values.
Column 'Model' has 0 null values.
Column 'Year' has 0 null values.
Column 'Status' has 0 null values.
Column 'Mileage' has 0 null values.
Column 'Price' has 0 null values.
Column 'MSRP' has 0 null values.
Total null values in the dataframe: Row(_c0=0, Model=0, Year=0, Status=0, Mileage=0, Price=0, MSRP=0)


In [0]:
df.show()

+---+--------------------+----+------+-------------+-------+---------------+
|_c0|               Model|Year|Status|      Mileage|  Price|           MSRP|
+---+--------------------+----+------+-------------+-------+---------------+
|  0|2022 Acura TLX A-...|2022|   New|Not available|$49,445|   MSRP $49,445|
|  1|2023 Acura RDX A-...|2023|   New|Not available|$50,895|  Not specified|
|  2|2023 Acura TLX Ty...|2023|   New|Not available|$57,745|  Not specified|
|  3|2023 Acura TLX Ty...|2023|   New|Not available|$57,545|  Not specified|
|  4|2019 Acura MDX Sp...|2019|  Used|   32,675 mi.|$40,990|$600 price drop|
|  5|2023 Acura TLX A-...|2023|   New|Not available|$50,195|   MSRP $50,195|
|  6|2023 Acura TLX A-...|2023|   New|Not available|$50,195|   MSRP $50,195|
|  7|2023 Acura TLX Ty...|2023|   New|Not available|$57,745|  Not specified|
|  8|2023 Acura TLX A-...|2023|   New|Not available|$47,995|  Not specified|
|  9|2022 Acura TLX A-...|2022|   New|Not available|$49,545|  Not specified|

In [0]:
df = df.drop('_c0')
df.show(10)

+--------------------+----+------+-------------+-------+---------------+
|               Model|Year|Status|      Mileage|  Price|           MSRP|
+--------------------+----+------+-------------+-------+---------------+
|2022 Acura TLX A-...|2022|   New|Not available|$49,445|   MSRP $49,445|
|2023 Acura RDX A-...|2023|   New|Not available|$50,895|  Not specified|
|2023 Acura TLX Ty...|2023|   New|Not available|$57,745|  Not specified|
|2023 Acura TLX Ty...|2023|   New|Not available|$57,545|  Not specified|
|2019 Acura MDX Sp...|2019|  Used|   32,675 mi.|$40,990|$600 price drop|
|2023 Acura TLX A-...|2023|   New|Not available|$50,195|   MSRP $50,195|
|2023 Acura TLX A-...|2023|   New|Not available|$50,195|   MSRP $50,195|
|2023 Acura TLX Ty...|2023|   New|Not available|$57,745|  Not specified|
|2023 Acura TLX A-...|2023|   New|Not available|$47,995|  Not specified|
|2022 Acura TLX A-...|2022|   New|Not available|$49,545|  Not specified|
+--------------------+----+------+-------------+---

In [0]:
element_count = df.groupBy('Mileage').count().orderBy('count', ascending=False)
element_count.show(7)

+-------------+-----+
|      Mileage|count|
+-------------+-----+
|Not available|47868|
|      310 mi.|  101|
|   23,000 mi.|   19|
|    1,000 mi.|   18|
|   29,000 mi.|   17|
|   35,000 mi.|   17|
|   26,000 mi.|   13|
+-------------+-----+
only showing top 7 rows



#Data Preparation

In [0]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *

In [0]:
#Clean up the Mileage column, mark the new car as 0 mi, then delete the characters and keep the numbers
df = df.withColumn('Mileage', when(col('Mileage') == 'Not available', '0 mi.').otherwise(col('Mileage')))
df = df.withColumn('Mileage', regexp_replace(col('Mileage'), ' mi.', ''))
df = df.withColumn('Mileage', regexp_replace(col('Mileage'), ',', ''))
df.show()

+--------------------+----+------+-------+-------+---------------+
|               Model|Year|Status|Mileage|  Price|           MSRP|
+--------------------+----+------+-------+-------+---------------+
|2022 Acura TLX A-...|2022|   New|      0|$49,445|   MSRP $49,445|
|2023 Acura RDX A-...|2023|   New|      0|$50,895|  Not specified|
|2023 Acura TLX Ty...|2023|   New|      0|$57,745|  Not specified|
|2023 Acura TLX Ty...|2023|   New|      0|$57,545|  Not specified|
|2019 Acura MDX Sp...|2019|  Used|  32675|$40,990|$600 price drop|
|2023 Acura TLX A-...|2023|   New|      0|$50,195|   MSRP $50,195|
|2023 Acura TLX A-...|2023|   New|      0|$50,195|   MSRP $50,195|
|2023 Acura TLX Ty...|2023|   New|      0|$57,745|  Not specified|
|2023 Acura TLX A-...|2023|   New|      0|$47,995|  Not specified|
|2022 Acura TLX A-...|2022|   New|      0|$49,545|  Not specified|
|2023 Acura Integr...|2023|   New|      0|$36,895|   MSRP $36,895|
|2023 Acura TLX A-...|2023|   New|      0|$48,395|   MSRP $48,

In [0]:
#Change Mileage data type
df = df.withColumn("Mileage", when(col("Mileage").rlike("^[0-9]+$"), col("Mileage")).otherwise("0"))
df = df.withColumn("Mileage", col("Mileage").cast(IntegerType()))
df.printSchema()
df.show()

root
 |-- Model: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Status: string (nullable = true)
 |-- Mileage: integer (nullable = true)
 |-- Price: string (nullable = true)
 |-- MSRP: string (nullable = true)

+--------------------+----+------+-------+-------+---------------+
|               Model|Year|Status|Mileage|  Price|           MSRP|
+--------------------+----+------+-------+-------+---------------+
|2022 Acura TLX A-...|2022|   New|      0|$49,445|   MSRP $49,445|
|2023 Acura RDX A-...|2023|   New|      0|$50,895|  Not specified|
|2023 Acura TLX Ty...|2023|   New|      0|$57,745|  Not specified|
|2023 Acura TLX Ty...|2023|   New|      0|$57,545|  Not specified|
|2019 Acura MDX Sp...|2019|  Used|  32675|$40,990|$600 price drop|
|2023 Acura TLX A-...|2023|   New|      0|$50,195|   MSRP $50,195|
|2023 Acura TLX A-...|2023|   New|      0|$50,195|   MSRP $50,195|
|2023 Acura TLX Ty...|2023|   New|      0|$57,745|  Not specified|
|2023 Acura TLX A-...|2023|   Ne

###Clean Price col

In [0]:
element_count_price = df.groupBy('Price').count().orderBy('count', ascending=False)
element_count_price.show(6)
price_count=df.select(col('Price')).count()
print('There are',price_count,'valid data in total')

+----------+-----+
|     Price|count|
+----------+-----+
|Not Priced|  652|
|   $29,995|  241|
|   $34,995|  226|
|   $72,010|  200|
|   $39,995|  192|
|   $24,995|  183|
+----------+-----+
only showing top 6 rows

There are 115762 valid data in total


In [0]:
#Since the proportion of not price data in the whole data is too small,  drop the not price data
df = df.filter(df.Price !='Not Priced')
df.select(col('Price')).count()


Out[12]: 115110

In [0]:
df.count()

Out[13]: 115110

In [0]:
len(df.columns)

Out[14]: 6

In [0]:
df = df.withColumn('Price', regexp_replace(col('Price'), ',', ''))
df.show()

+--------------------+----+------+-------+------+---------------+
|               Model|Year|Status|Mileage| Price|           MSRP|
+--------------------+----+------+-------+------+---------------+
|2022 Acura TLX A-...|2022|   New|      0|$49445|   MSRP $49,445|
|2023 Acura RDX A-...|2023|   New|      0|$50895|  Not specified|
|2023 Acura TLX Ty...|2023|   New|      0|$57745|  Not specified|
|2023 Acura TLX Ty...|2023|   New|      0|$57545|  Not specified|
|2019 Acura MDX Sp...|2019|  Used|  32675|$40990|$600 price drop|
|2023 Acura TLX A-...|2023|   New|      0|$50195|   MSRP $50,195|
|2023 Acura TLX A-...|2023|   New|      0|$50195|   MSRP $50,195|
|2023 Acura TLX Ty...|2023|   New|      0|$57745|  Not specified|
|2023 Acura TLX A-...|2023|   New|      0|$47995|  Not specified|
|2022 Acura TLX A-...|2022|   New|      0|$49545|  Not specified|
|2023 Acura Integr...|2023|   New|      0|$36895|   MSRP $36,895|
|2023 Acura TLX A-...|2023|   New|      0|$48395|   MSRP $48,395|
|2023 Acur

##Change Price data type

In [0]:
df = df.withColumn('price', regexp_replace(col('price'), '[$€£¥]', ''))

df.printSchema()
df.show(5)

root
 |-- Model: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Status: string (nullable = true)
 |-- Mileage: integer (nullable = true)
 |-- price: string (nullable = true)
 |-- MSRP: string (nullable = true)

+--------------------+----+------+-------+-----+---------------+
|               Model|Year|Status|Mileage|price|           MSRP|
+--------------------+----+------+-------+-----+---------------+
|2022 Acura TLX A-...|2022|   New|      0|49445|   MSRP $49,445|
|2023 Acura RDX A-...|2023|   New|      0|50895|  Not specified|
|2023 Acura TLX Ty...|2023|   New|      0|57745|  Not specified|
|2023 Acura TLX Ty...|2023|   New|      0|57545|  Not specified|
|2019 Acura MDX Sp...|2019|  Used|  32675|40990|$600 price drop|
+--------------------+----+------+-------+-----+---------------+
only showing top 5 rows



In [0]:
df = df.withColumn('price', col('price').cast('integer'))
df.printSchema()
df.show(5)

root
 |-- Model: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Status: string (nullable = true)
 |-- Mileage: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- MSRP: string (nullable = true)

+--------------------+----+------+-------+-----+---------------+
|               Model|Year|Status|Mileage|price|           MSRP|
+--------------------+----+------+-------+-----+---------------+
|2022 Acura TLX A-...|2022|   New|      0|49445|   MSRP $49,445|
|2023 Acura RDX A-...|2023|   New|      0|50895|  Not specified|
|2023 Acura TLX Ty...|2023|   New|      0|57745|  Not specified|
|2023 Acura TLX Ty...|2023|   New|      0|57545|  Not specified|
|2019 Acura MDX Sp...|2019|  Used|  32675|40990|$600 price drop|
+--------------------+----+------+-------+-----+---------------+
only showing top 5 rows



##Status col understanding and cleaning

In [0]:
element_count_Status = df.groupBy('Status').count().orderBy('Status', ascending=False)
element_count_Status.show()


+--------------------+-----+
|              Status|count|
+--------------------+-----+
|Volkswagen Certified|  797|
|                Used|61627|
|    Toyota Certified|  183|
|   Porsche Certified| 1914|
|                 New|47434|
|  INFINITI Certified|  865|
|      Ford Certified|   29|
|     Dodge Certified|  388|
| Chevrolet Certified|  211|
|       BMW Certified|  609|
|     Acura Certified| 1053|
+--------------------+-----+



In [0]:
from pyspark.sql.functions import when, col
df = df.withColumn('Status', when(col('Status').isin(['Used', 'New']), col('Status')).otherwise('Certified'))

element_count_Status = df.groupBy('Status').count().orderBy('Status', ascending=False)
element_count_Status.show()

+---------+-----+
|   Status|count|
+---------+-----+
|     Used|61627|
|      New|47434|
|Certified| 6049|
+---------+-----+



##Extract the brand from the car model into a new column

In [0]:
from pyspark.sql.functions import split
df = df.withColumn('Brand', split(df['Model'], ' ').getItem(1))
df.show()

+--------------------+----+------+-------+-----+---------------+-----+
|               Model|Year|Status|Mileage|price|           MSRP|Brand|
+--------------------+----+------+-------+-----+---------------+-----+
|2022 Acura TLX A-...|2022|   New|      0|49445|   MSRP $49,445|Acura|
|2023 Acura RDX A-...|2023|   New|      0|50895|  Not specified|Acura|
|2023 Acura TLX Ty...|2023|   New|      0|57745|  Not specified|Acura|
|2023 Acura TLX Ty...|2023|   New|      0|57545|  Not specified|Acura|
|2019 Acura MDX Sp...|2019|  Used|  32675|40990|$600 price drop|Acura|
|2023 Acura TLX A-...|2023|   New|      0|50195|   MSRP $50,195|Acura|
|2023 Acura TLX A-...|2023|   New|      0|50195|   MSRP $50,195|Acura|
|2023 Acura TLX Ty...|2023|   New|      0|57745|  Not specified|Acura|
|2023 Acura TLX A-...|2023|   New|      0|47995|  Not specified|Acura|
|2022 Acura TLX A-...|2022|   New|      0|49545|  Not specified|Acura|
|2023 Acura Integr...|2023|   New|      0|36895|   MSRP $36,895|Acura|
|2023 

In [0]:
element_count_Brand = df.groupBy('Brand').count().orderBy('Brand', ascending=False)
element_count_Brand.show()

+-------------+-----+
|        Brand|count|
+-------------+-----+
|   Volkswagen| 9968|
|       Toyota| 5709|
|        Tesla| 9068|
|      Porsche| 9961|
|Mercedes-Benz|10100|
|        Lexus| 9965|
|     INFINITI| 8664|
|      Hyundai| 8280|
|         Ford| 4465|
|        Dodge| 9819|
|    Chevrolet| 9914|
|          BMW| 9827|
|        Acura| 9370|
+-------------+-----+



In [0]:
df.select('Mileage','price').describe().show()

+-------+-----------------+-----------------+
|summary|          Mileage|            price|
+-------+-----------------+-----------------+
|  count|           115110|           115110|
|   mean| 28279.3212579272|51517.98868908001|
| stddev|38055.55058284502|37931.87068346887|
|    min|                0|             1800|
|    max|           974302|          2499900|
+-------+-----------------+-----------------+



##Clean outliers

In [0]:
quantiles = {
    c: dict(
        zip(["q1", "q3"], df.approxQuantile(c, [0.25, 0.75], 0))
    )
    for c in ["price"]
}
quantiles

Out[23]: {'price': {'q1': 29980.0, 'q3': 61280.0}}

In [0]:
for i in quantiles:
    iqr = quantiles[i]['q3'] - quantiles[i]['q1']
    quantiles[i]['lower_bound'] = quantiles[i]['q1'] - (iqr * 1.5)
    quantiles[i]['upper_bound'] = quantiles[i]['q3'] + (iqr * 1.5)
print(quantiles)

{'price': {'q1': 29980.0, 'q3': 61280.0, 'lower_bound': -16970.0, 'upper_bound': 108230.0}}


In [0]:
import pyspark.sql.functions as f
df_clean=df.select(
    "*",
    *[
        f.when(
            f.col(c).between(quantiles[c]['lower_bound'], quantiles[c]['upper_bound']),
            0
        ).otherwise(1).alias(c+"_out") 
        for c in ["price"]
    ]
)
df_clean.show(10)

+--------------------+----+------+-------+-----+---------------+-----+---------+
|               Model|Year|Status|Mileage|price|           MSRP|Brand|price_out|
+--------------------+----+------+-------+-----+---------------+-----+---------+
|2022 Acura TLX A-...|2022|   New|      0|49445|   MSRP $49,445|Acura|        0|
|2023 Acura RDX A-...|2023|   New|      0|50895|  Not specified|Acura|        0|
|2023 Acura TLX Ty...|2023|   New|      0|57745|  Not specified|Acura|        0|
|2023 Acura TLX Ty...|2023|   New|      0|57545|  Not specified|Acura|        0|
|2019 Acura MDX Sp...|2019|  Used|  32675|40990|$600 price drop|Acura|        0|
|2023 Acura TLX A-...|2023|   New|      0|50195|   MSRP $50,195|Acura|        0|
|2023 Acura TLX A-...|2023|   New|      0|50195|   MSRP $50,195|Acura|        0|
|2023 Acura TLX Ty...|2023|   New|      0|57745|  Not specified|Acura|        0|
|2023 Acura TLX A-...|2023|   New|      0|47995|  Not specified|Acura|        0|
|2022 Acura TLX A-...|2022| 

In [0]:
from pyspark.sql.functions import col
df_clean=df_clean.withColumn("outliers", col("price_out"))
df_clean.show()

+--------------------+----+------+-------+-----+---------------+-----+---------+--------+
|               Model|Year|Status|Mileage|price|           MSRP|Brand|price_out|outliers|
+--------------------+----+------+-------+-----+---------------+-----+---------+--------+
|2022 Acura TLX A-...|2022|   New|      0|49445|   MSRP $49,445|Acura|        0|       0|
|2023 Acura RDX A-...|2023|   New|      0|50895|  Not specified|Acura|        0|       0|
|2023 Acura TLX Ty...|2023|   New|      0|57745|  Not specified|Acura|        0|       0|
|2023 Acura TLX Ty...|2023|   New|      0|57545|  Not specified|Acura|        0|       0|
|2019 Acura MDX Sp...|2019|  Used|  32675|40990|$600 price drop|Acura|        0|       0|
|2023 Acura TLX A-...|2023|   New|      0|50195|   MSRP $50,195|Acura|        0|       0|
|2023 Acura TLX A-...|2023|   New|      0|50195|   MSRP $50,195|Acura|        0|       0|
|2023 Acura TLX Ty...|2023|   New|      0|57745|  Not specified|Acura|        0|       0|
|2023 Acur

In [0]:
# dropping outliers
df_clean = df_clean.filter((df_clean.outliers == 0) )
df_clean=df_clean.select(["Model","Year", "Status", "Mileage","price","MSRP", "Brand"])
df.select('price','Mileage').describe().show()

+-------+-----------------+-----------------+
|summary|            price|          Mileage|
+-------+-----------------+-----------------+
|  count|           115110|           115110|
|   mean|51517.98868908001| 28279.3212579272|
| stddev|37931.87068346887|38055.55058284502|
|    min|             1800|                0|
|    max|          2499900|           974302|
+-------+-----------------+-----------------+



In [0]:
import numpy as np
print("proportion of the lost Rows: ",np.round((df.count()-df_clean.count())/df.count(),4))
#We only delete a small part of the data, which does not constitute a mailbox for subsequent processing

proportion of the lost Rows:  0.0712


#Data Visualization

In [0]:
df_clean.registerTempTable("dataclean")
display(sqlContext.sql("select * from dataclean"))
#Due to a migration quirk in Pyspark, all our Visualizations are in a seprate file; 'Predicting Cars Sales Final Project V2.(Visuals).ipynb'

/databricks/spark/python/pyspark/sql/dataframe.py:234: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


Model,Year,Status,Mileage,price,MSRP,Brand
2022 Acura TLX A-Spec,2022,New,0,49445,"MSRP $49,445",Acura
2023 Acura RDX A-Spec,2023,New,0,50895,Not specified,Acura
2023 Acura TLX Type S,2023,New,0,57745,Not specified,Acura
2023 Acura TLX Type S,2023,New,0,57545,Not specified,Acura
2019 Acura MDX Sport Hybrid 3.0L w/Technology Package,2019,Used,32675,40990,$600 price drop,Acura
2023 Acura TLX A-Spec,2023,New,0,50195,"MSRP $50,195",Acura
2023 Acura TLX A-Spec,2023,New,0,50195,"MSRP $50,195",Acura
2023 Acura TLX Type S,2023,New,0,57745,Not specified,Acura
2023 Acura TLX A-Spec,2023,New,0,47995,Not specified,Acura
2022 Acura TLX A-Spec,2022,New,0,49545,Not specified,Acura


<div style="background-color:#F7CAC9; color:#4A3C31; padding:10px;">

# Modeling

</div>

In [0]:
df_clean2 = df.withColumn('Year', col('price').cast('string'))


##Linear Regression

In [0]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

#Explore the relationship between vehicle age and mileage by first processing the data. Create a list of ages
df_clean2 = df_clean2.withColumn("Status", when(df.Status == "new", 0).otherwise(1))
df_clean2 = df_clean2.withColumn("age", lit(2023) - year("Year"))
df_clean2.show()

+--------------------+-----+------+-------+-----+---------------+-----+------+
|               Model| Year|Status|Mileage|price|           MSRP|Brand|   age|
+--------------------+-----+------+-------+-----+---------------+-----+------+
|2022 Acura TLX A-...|49445|     1|      0|49445|   MSRP $49,445|Acura|-47422|
|2023 Acura RDX A-...|50895|     1|      0|50895|  Not specified|Acura|-48872|
|2023 Acura TLX Ty...|57745|     1|      0|57745|  Not specified|Acura|-55722|
|2023 Acura TLX Ty...|57545|     1|      0|57545|  Not specified|Acura|-55522|
|2019 Acura MDX Sp...|40990|     1|  32675|40990|$600 price drop|Acura|-38967|
|2023 Acura TLX A-...|50195|     1|      0|50195|   MSRP $50,195|Acura|-48172|
|2023 Acura TLX A-...|50195|     1|      0|50195|   MSRP $50,195|Acura|-48172|
|2023 Acura TLX Ty...|57745|     1|      0|57745|  Not specified|Acura|-55722|
|2023 Acura TLX A-...|47995|     1|      0|47995|  Not specified|Acura|-45972|
|2022 Acura TLX A-...|49545|     1|      0|49545|  N

In [0]:
# Transform the features to vectors.
assembler = VectorAssembler(inputCols=["age", "Mileage", "Status"], outputCol="features")
df_clean2 = assembler.transform(df_clean2)

In [0]:
#The data presentation shows that each vehicle has a vector of selected feature columns. It is used to bring in to build the model.
df_clean2.show(10)

+--------------------+-----+------+-------+-----+---------------+-----+------+--------------------+
|               Model| Year|Status|Mileage|price|           MSRP|Brand|   age|            features|
+--------------------+-----+------+-------+-----+---------------+-----+------+--------------------+
|2022 Acura TLX A-...|49445|     1|      0|49445|   MSRP $49,445|Acura|-47422|  [-47422.0,0.0,1.0]|
|2023 Acura RDX A-...|50895|     1|      0|50895|  Not specified|Acura|-48872|  [-48872.0,0.0,1.0]|
|2023 Acura TLX Ty...|57745|     1|      0|57745|  Not specified|Acura|-55722|  [-55722.0,0.0,1.0]|
|2023 Acura TLX Ty...|57545|     1|      0|57545|  Not specified|Acura|-55522|  [-55522.0,0.0,1.0]|
|2019 Acura MDX Sp...|40990|     1|  32675|40990|$600 price drop|Acura|-38967|[-38967.0,32675.0...|
|2023 Acura TLX A-...|50195|     1|      0|50195|   MSRP $50,195|Acura|-48172|  [-48172.0,0.0,1.0]|
|2023 Acura TLX A-...|50195|     1|      0|50195|   MSRP $50,195|Acura|-48172|  [-48172.0,0.0,1.0]|


In [0]:
train_data,test_data = df_clean2.randomSplit([0.7,0.3])
#Creating multiple regression models
lr = LinearRegression(featuresCol= 'features',labelCol = "price",maxIter=10, regParam=0.1, elasticNetParam=0.8)
model = lr.fit(train_data)

In [0]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(model.coefficients,model.intercept))

Coefficients: [-0.9999977231189012,0.0,0.0] Intercept: 2023.1125874909812


+ The coefficient of "status" is 0.
+ The 'status' feature is not significantly contributing to the prediction of car price in the model.

In [0]:
predictions = model.transform(test_data)
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")
r2_score = evaluator.evaluate(predictions)

print("R2 score on test data: {:.3f}".format(r2_score))

Root Mean Squared Error (RMSE) on test data = 0.087894
R2 score on test data: 1.000


In [0]:
from pyspark.ml.feature import VectorAssembler, PolynomialExpansion
# Then, let's apply a polynomial expansion to create polynomial features
poly_expansion = PolynomialExpansion(inputCol="features", outputCol="poly_features", degree=2)
df_expanded = poly_expansion.transform(df_clean2)

In [0]:
train_data, test_data = df_expanded.randomSplit([0.7, 0.3], seed=1234)
# Define the linear regression model
lr = LinearRegression(featuresCol="poly_features", labelCol="price", maxIter=10, regParam=0.1)

# Fit the model to the training data
lr_model = lr.fit(train_data)

# Make predictions on the test data
predictions = lr_model.transform(test_data)

# Evaluate the model using R-squared
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="price", metricName="r2")
r2_score = evaluator.evaluate(predictions)

# Print the R-squared score
print("R-squared score on test data: {:.3f}".format(r2_score))

R-squared score on test data: 1.000


The results of the linear regression model were not satisfactory, and this model explained 38% of the variability in the dependent variable. In other words, the model did not fit very well and 62% of the variability in the dependent variable was not explained by the model. there was no linear relationship between Status and the dependent variable

We continued with other machine learning models to train the fitted data

##Random Forest

In [0]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col

In [0]:
df1=df_clean
modelIndexer = StringIndexer(inputCol="Model", outputCol="ModelIndex")
statusIndexer = StringIndexer(inputCol="Status", outputCol="StatusIndex")
brandIndexer = StringIndexer(inputCol="Brand", outputCol="BrandIndex")
indexed = modelIndexer.fit(df_clean).transform(df_clean)
indexed = statusIndexer.fit(indexed).transform(indexed)
indexed = brandIndexer.fit(indexed).transform(indexed)
assembler = VectorAssembler(inputCols=["Year", "Mileage", "StatusIndex", "BrandIndex","ModelIndex"], outputCol="features")


In [0]:

data = assembler.transform(indexed).select("features", "price")

In [0]:
(trainingData, testData) = data.randomSplit([0.7, 0.3], seed = 1)
rf = RandomForestRegressor(featuresCol="features", labelCol="price", numTrees=10,maxBins=9000)
model = rf.fit(trainingData)
predictions = model.transform(testData)

<div style="background-color:#F7CAC9; color:#4A3C31; padding:10px;">
    
# Evaluation -- Random Forest

</div>

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")

rmse = evaluator.evaluate(predictions)
print("RMSE = %g" % rmse)

evaluator2 = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")

r2 = evaluator2.evaluate(predictions)
print("R-squared = %g" % r2)

RMSE = 11262
R-squared = 0.704623


In [0]:
importances = model.featureImportances.toArray()

# Create a dictionary of feature names and their corresponding importance scores
feature_importances = {}
for i in range(len(["Year", "Mileage", "StatusIndex", "BrandIndex","ModelIndex"])):
    feature_importances[["Year", "Mileage", "StatusIndex", "BrandIndex","ModelIndex"][i]] = importances[i]

# Print the feature importance scores in descending order
print("Feature Importance Scores:")
for feature, score in sorted(feature_importances.items(), key=lambda x: x[1], reverse=True):
    print("{}: {}".format(feature, score))

Feature Importance Scores:
ModelIndex: 0.6724656255921914
Mileage: 0.09118801729185302
BrandIndex: 0.08045864949532205
Year: 0.07885011075110157
StatusIndex: 0.07703759686953197


#H2O XGBoost

In [0]:
from pysparkling import *
from pyspark.sql import SparkSession
import h2o
from h2o.estimators import H2OXGBoostEstimator
from h2o.automl import H2OAutoML
from h2o.estimators import H2OGradientBoostingEstimator

In [0]:
# Initialize H2O
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,15 mins 18 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.3
H2O_cluster_version_age:,"14 days, 11 hours and 59 minutes"
H2O_cluster_name:,H2O_from_python_root_zvwch3
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.272 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [0]:
num_rows = df_clean.count()

print("Number of rows:", num_rows)

Number of rows: 106911


##Due to the large amount of data, there is a lot of pressure on subsequent modeling, so 10% of the data is randomly sampled for modeling

In [0]:
sampled_df = df_clean.sample(fraction=0.1, seed=42)
sampled_df_1 = df_clean.sample(fraction=0.1, seed=44)
num_rows = sampled_df.count()

print("Number of rows:", num_rows)

Number of rows: 10696


In [0]:
sampled_df = sampled_df.drop('MSRP')

df2=sampled_df

In [0]:
df2 = df2.withColumn("Year", col("Year").cast(IntegerType()))

In [0]:
h2o_frame = h2o.H2OFrame(df2.toPandas())

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [0]:
train, valid = h2o_frame.split_frame(ratios=[0.7])

# Define the predictor and response variables
predictors = h2o_frame.col_names[:-1]
response = "price"


In [0]:
aml = H2OAutoML(max_models=10, seed=1, include_algos=["XGBoost"])
aml.train(x=predictors, y=response, training_frame=train, validation_frame=valid)


AutoML progress: |
23:51:25.329: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_3_AutoML_2_20230418_235125
 
 
 
 
 
 Model Summary: 
 
 number_of_trees 
 
 146.0 
 
 
 
 ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 127657336.46599188
RMSE: 11298.554618445312
MAE: 8342.326995217987
RMSLE: 0.2806821593323781
Mean Residual Deviance: 127657336.46599188 
 ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 153281918.3994111
RMSE: 12380.707508030837
MAE: 9028.690536265884
RMSLE: 0.2998241769473745
Mean Residual Deviance: 153281918.3994111 
 ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 168587715.218318
RMSE: 12984.133210126813
MAE: 9289.034692027473
RMSLE: 0.3092280687298753
Mean Residual Deviance: 168587715.218318 
 
 
 
 
 Cross-Validation Metrics Summary: 
 
 mean 
 sd 
 cv_1_valid 
 cv_2_valid 
 cv_3_valid 
 cv_4_valid 
 cv_5_valid 
 mae 
 9289.056 
 129.01039 
 9352.332 
 9113.777 
 9212.345 
 9319.547 
 9447.277 
 mean_residual_deviance 
 168588432.0000000 
 5997677.5 
 170850176.0000000 
 159048768.0000000 
 166626368.0000000 
 172424464.0000000 
 173992416.0000000 
 mse 
 168588432.0000000 
 5997677.5 
 170850176.0000000 
 159048768.0000000 
 166626368.0000000 
 172424464.0000000 
 173992416.0000000 
 r2 
 0.6024725 
 0.0094799 
 0.596629 
 0.5985825 
 0.6189855 
 0.5963867 
 0.6017790 
 residual_deviance 
 168588432.0000000 
 5997677.5 
 170850176.0000000 
 159048768.0000000 
 166626368.0000000 
 172424464.0000000 
 173992416.0000000 
 rmse 
 12982.494 
 232.58072 
 13070.967 
 12611.454 
 12908.384 
 13131.05 
 13190.618 
 rmsle 
 0.3092017 
 0.0045286 
 0.3147016 
 0.3041806 
 0.3114903 
 0.3048596 
 0.3107767 
 
 
 
 
 
 
 
 Scoring History: 
 
 timestamp 
 duration 
 number_of_trees 
 training_rmse 
 training_mae 
 training_deviance 
 validation_rmse 
 validation_mae 
 validation_deviance 
 
 2023-04-18 23:51:48 
 11.458 sec 
 0.0 
 48538.4440949 
 43950.0959407 
 2355980555.1491857 
 48662.7353789 
 44171.5629872 
 2368061814.5593233 
 
 2023-04-18 23:51:48 
 11.515 sec 
 5.0 
 17106.0855957 
 12113.9382142 
 292618164.4063267 
 17373.4969137 
 12267.3395630 
 301838395.0087339 
 
 2023-04-18 23:51:48 
 11.558 sec 
 10.0 
 14742.6435992 
 11039.3457896 
 217345540.2943621 
 15054.9804032 
 11252.5870057 
 226652434.9414198 
 
 2023-04-18 23:51:48 
 11.630 sec 
 15.0 
 14351.7934946 
 10946.0547089 
 205973976.5128312 
 14670.5963553 
 11164.5953280 
 215226397.4200891 
 
 2023-04-18 23:51:48 
 11.716 sec 
 20.0 
 14075.4041897 
 10776.9051721 
 198117003.1031882 
 14437.6754017 
 11023.6485112 
 208446471.0041135 
 
 2023-04-18 23:51:49 
 11.761 sec 
 25.0 
 13836.2987095 
 10601.1657471 
 191443161.9794477 
 14242.0082017 
 10877.2478310 
 202834797.6164260 
 
 2023-04-18 23:51:49 
 11.814 sec 
 30.0 
 13629.6717458 
 10464.8706613 
 185767951.8988010 
 14123.9969134 
 10789.2652316 
 199487288.8108598 
 
 2023-04-18 23:51:49 
 11.868 sec 
 35.0 
 13443.9243427 
 10325.6604036 
 180739101.7331050 
 13947.3721785 
 10643.2954179 
 194529190.6860192 
 
 2023-04-18 23:51:49 
 11.923 sec 
 40.0 
 13276.3214107 
 10196.4800562 
 176260710.1991616 
 13824.6059321 
 10539.2923378 
 191119729.1777047 
 
 2023-04-18 23:51:49 
 11.979 sec 
 45.0 
 13085.4569475 
 10031.0162700 
 171229183.5246313 
 13636.6002298 
 10383.7273956 
 185956865.8261668 
 --- 
 --- 
 --- 
 --- 
 --- 
 --- 
 --- 
 --- 
 --- 
 --- 
 
 2023-04-18 23:51:50 
 12.820 sec 
 105.0 
 11858.3246866 
 8904.2821852 
 140619864.3726806 
 12713.9913066 
 9432.8720140 
 161645574.9443339 
 
 2023-04-18 23:51:50 
 12.901 sec 
 110.0 
 11784.4824646 
 8825.9995299 
 138874026.9589401 
 12673.2552504 
 9387.7194708 
 160611398.6409201 
 
 2023-04-18 23:51:50 
 12.980 sec 
 115.0 
 11718.3138853 
 8756.3273268 
 137318880.3144288 
 12635.5703943 
 9337.0372053 
 159657639.1885911 
 
 2023-04-18 23:51:50 
 13.060 sec 
 120.0 
 11654.7054640 
 8690.6642700 
 135832159.4521349 
 12585.9972818 
 928

##XGBoost model evaluation

In [0]:
# View the leaderboard
lb = aml.leaderboard
print(lb.head(rows=lb.nrows))

model_id                                            rmse          mse       mae     rmsle    mean_residual_deviance
XGBoost_3_AutoML_2_20230418_235125               12984.1  1.68588e+08   9289.03  0.309228               1.68588e+08
XGBoost_grid_1_AutoML_2_20230418_235125_model_2  13262.7  1.759e+08     9407.54  0.313923               1.759e+08
XGBoost_grid_1_AutoML_2_20230418_235125_model_3  13675.2  1.8701e+08    9605.91  0.321363               1.8701e+08
XGBoost_grid_1_AutoML_2_20230418_235125_model_1  14109.4  1.99077e+08   9933.82  0.330016               1.99077e+08
XGBoost_2_AutoML_2_20230418_235125               14238.5  2.02735e+08  10145.5   0.333349               2.02735e+08
XGBoost_grid_1_AutoML_2_20230418_235125_model_7  14265.8  2.03513e+08  10384.2   0.332363               2.03513e+08
XGBoost_grid_1_AutoML_2_20230418_235125_model_4  15289.4  2.33765e+08  11011.8   0.351939               2.33765e+08
XGBoost_grid_1_AutoML_2_20230418_235125_model_6  15415.1  2.37626e+08  1119

In [0]:
# Get the best XGBoost model
best_model = h2o.get_model(lb[0,"model_id"])

In [0]:
# Calculate feature importance
feature_importance = best_model.varimp(use_pandas=True)
print(feature_importance)

                                          variable  relative_importance  \
0                                             Year         1.263253e+12   
1                                          Mileage         6.615109e+11   
2          Model.2023 Mercedes-Benz GLS 450 4MATIC         8.547312e+10   
3      Model.2023 Mercedes-Benz GLE 450 AWD 4MATIC         7.527318e+10   
4                                       Status.New         6.625506e+10   
..                                             ...                  ...   
290        Model.2019 Volkswagen Jetta 1.4T R-Line         6.266694e+08   
291  Model.2023 Volkswagen Jetta GLI 2.0T Autobahn         6.195970e+08   
292                      Model.2014 Dodge Dart SXT         4.843110e+08   
293                 Model.2013 Porsche Cayenne GTS         4.392428e+08   
294                  Model.2013 INFINITI JX35 Base         1.699840e+08   

     scaled_importance  percentage  
0             1.000000    0.344959  
1             0.523657   

##H2O Stacked Ensemble Model

In [0]:
import h2o
from h2o.estimators import H2OGradientBoostingEstimator, H2OXGBoostEstimator, H2OStackedEnsembleEstimator
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [0]:
target_col = "price"

sampled_df_1 = sampled_df_1.withColumn("Year", col("Year").cast(IntegerType()))
h2o_frame = h2o.H2OFrame(sampled_df_1.toPandas())

# Split data into training and validation sets
train, valid = h2o_frame.split_frame(ratios=[0.8], seed=123)

# Specify the predictor columns
predictors = ["Model", "Year", "Status", "Mileage", "Brand"]


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [0]:
gbm = H2OGradientBoostingEstimator(seed=123, ntrees=50, max_depth=5, nfolds=5, keep_cross_validation_predictions=True)
gbm.train(x=predictors, y=target_col, training_frame=train, validation_frame=valid)

xgb = H2OXGBoostEstimator(seed=123, ntrees=50, max_depth=5, nfolds=5, keep_cross_validation_predictions=True)
xgb.train(x=predictors, y=target_col, training_frame=train, validation_frame=valid)


gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_model_python_1681860957017_13
 
 
 
 
 
 Model Summary: 
 
 number_of_trees 
 
 50.0 
 
 
 
 ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 98363472.38393666
RMSE: 9917.83607365723
MAE: 7522.321629041551
RMSLE: 0.24095875457177443
Mean Residual Deviance: 98363472.38393666 
 ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 112550779.15379241
RMSE: 10608.995200007983
MAE: 7935.851415129537
RMSLE: 0.24917038467555455
Mean Residual Deviance: 112550779.15379241 
 ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 115057463.92638953
RMSE: 10726.484229531572
MAE: 7954.442715198111
RMSLE: 0.2571384104568273
Mean Residual Deviance: 115057463.92638953 
 
 
 
 
 Cross-Validation Metrics Summary: 
 
 mean 
 sd 
 cv_1_valid 
 cv_2_valid 
 cv_3_valid 
 cv_4_valid 
 cv_5_valid 
 mae 
 7951.7744 
 106.5446240 
 8065.7354 
 8041.022 
 7939.466 
 7911.4688 
 7801.1797 
 mean_residual_deviance 
 114961304.0000000 
 3508475.2 
 118576824.0000000 
 116271944.0000000 
 116867632.0000000 
 113477984.0000000 
 109612144.0000000 
 mse 
 114961304.0000000 
 3508475.2 
 118576824.0000000 
 116271944.0000000 
 116867632.0000000 
 113477984.0000000 
 109612144.0000000 
 r2 
 0.7276436 
 0.0114109 
 0.7360398 
 0.7351786 
 0.7095104 
 0.7231623 
 0.7343270 
 residual_deviance 
 114961304.0000000 
 3508475.2 
 118576824.0000000 
 116271944.0000000 
 116867632.0000000 
 113477984.0000000 
 109612144.0000000 
 rmse 
 10720.993 
 164.36778 
 10889.299 
 10782.947 
 10810.533 
 10652.6045000 
 10469.582 
 rmsle 
 0.2571628 
 0.0081569 
 0.2511663 
 0.2657568 
 0.2618524 
 0.2461781 
 0.2608604 
 
 
 
 
 
 
 
 Scoring History: 
 
 timestamp 
 duration 
 number_of_trees 
 training_rmse 
 training_mae 
 training_deviance 
 validation_rmse 
 validation_mae 
 validation_deviance 
 
 2023-04-18 23:53:01 
 8.684 sec 
 0.0 
 48481.4792440 
 43898.0658577 
 2350453829.6826105 
 48404.8591092 
 43829.8619866 
 2343030385.3808498 
 
 2023-04-18 23:53:01 
 8.724 sec 
 1.0 
 35379.3426095 
 30769.8974651 
 1251697883.4807744 
 35394.1770448 
 30761.0130211 
 1252747768.6777210 
 
 2023-04-18 23:53:01 
 8.758 sec 
 2.0 
 26517.9608651 
 21632.3215542 
 703202248.4405608 
 26614.8826707 
 21663.1522185 
 708351979.5776241 
 
 2023-04-18 23:53:01 
 8.784 sec 
 3.0 
 20664.3237444 
 15707.9386486 
 427014275.8137801 
 20833.6715037 
 15832.9533344 
 434041868.3220040 
 
 2023-04-18 23:53:01 
 8.825 sec 
 4.0 
 16992.2716018 
 12275.6179794 
 288737294.1894515 
 17249.8802564 
 12483.8341821 
 297558368.8586919 
 
 2023-04-18 23:53:01 
 8.846 sec 
 5.0 
 14740.1424474 
 10381.7861854 
 217271799.3691976 
 15012.8542711 
 10614.9280384 
 225385793.3653544 
 
 2023-04-18 23:53:01 
 8.865 sec 
 6.0 
 13447.9324875 
 9469.5057706 
 180846888.1893408 
 13764.9589014 
 9696.0448370 
 189474093.5575885 
 
 2023-04-18 23:53:01 
 8.887 sec 
 7.0 
 12652.0868786 
 9010.0291222 
 160075302.3845759 
 13048.8891762 
 9288.7135976 
 170273508.7334970 
 
 2023-04-18 23:53:01 
 8.907 sec 
 8.0 
 12172.9704441 
 8769.5263628 
 148181209.4340629 
 12595.8927584 
 9045.7770653 
 158656514.3805833 
 
 2023-04-18 23:53:01 
 8.929 sec 
 9.0 
 11930.5120596 
 8680.2956129 
 142337118.0051706 
 12385.8261414 
 8976.4765009 
 153408689.2061209 
 --- 
 --- 
 --- 
 --- 
 --- 
 --- 
 --- 
 --- 
 --- 
 --- 
 
 2023-04-18 23:53:02 
 10.003 sec 
 41.0 
 10172.5053791 
 7701.8819558 
 103479865.6886934 
 10864.7317108 
 8111.9588351 
 118042395.1471028 
 
 2023-04-18 23:53:02 
 10.038 sec 
 42.0 
 10129.4416572 
 7675.2010308 
 102605588.2873019 
 10831.6695258 
 8093.6688977 
 117325064.7158096 
 
 2023-04-18 23:53:02 
 10.073 sec 
 43.0 
 10103.8031206 
 7657.1036222 
 102086837.4991076 
 10770.6188181 
 8066.5113214 
 116006229.7249334 
 
 2023-04-18 23:53:02 
 10.109 sec 
 44.0 
 10078.2517467 
 7642.8862763 
 101571158.2689638 
 10771.0321985 
 8065.3824012 

In [0]:
stack = H2OStackedEnsembleEstimator(base_models=[gbm, xgb], seed=123)
stack.train(x=predictors, y=target_col, training_frame=train, validation_frame=valid)

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),1/2
# GBM base models (used / total),1/1
# XGBoost base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,AUTO
Metalearner nfolds,0
Metalearner fold_column,None
Custom metalearner hyperparameters,None


In [0]:
predictions = stack.predict(valid)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


##Stacked Ensemble Model Evaluation

In [0]:
stacked_ensemble_perf = stack.model_performance(test_data=valid)
mse = stacked_ensemble_perf.mse()
rmse = stacked_ensemble_perf.rmse()
mae = stacked_ensemble_perf.mae()
r2 = stacked_ensemble_perf.r2()

print("Stacked Ensemble Model Performance:")
print("MSE: {:.4f}".format(mse))
print("RMSE: {:.4f}".format(rmse))
print("MAE: {:.4f}".format(mae))
print("R^2: {:.4f}".format(r2))

Stacked Ensemble Model Performance:
MSE: 65288968.2084
RMSE: 8080.1589
MAE: 5448.0254
R^2: 0.8453
